In [2]:
import os

import tensorflow as tf
from tensorflow import keras

In [6]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0


In [7]:
def create_model():
    model = tf.keras.models.Sequential([
        keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10)
    ])
    
    model.compile(optimizer='adam',
                 loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                 metrics=[tf.metrics.SparseCategoricalAccuracy()])
    
    return model
model = create_model()
 
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [8]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

model.fit(train_images,
         train_labels,
         epochs=10,
         validation_data=(test_images, test_labels),
         callbacks=[cp_callback])

Epoch 1/10
32/32 [==============================] - 2s 34ms/step - loss: 1.6436 - sparse_categorical_accuracy: 0.5152 - val_loss: 0.7255 - val_sparse_categorical_accuracy: 0.7960

Epoch 00001: saving model to training_1\cp.ckpt
Epoch 2/10
32/32 [==============================] - 0s 8ms/step - loss: 0.4340 - sparse_categorical_accuracy: 0.8983 - val_loss: 0.5142 - val_sparse_categorical_accuracy: 0.8430

Epoch 00002: saving model to training_1\cp.ckpt
Epoch 3/10
32/32 [==============================] - 0s 6ms/step - loss: 0.2735 - sparse_categorical_accuracy: 0.9336 - val_loss: 0.4637 - val_sparse_categorical_accuracy: 0.8580

Epoch 00003: saving model to training_1\cp.ckpt
Epoch 4/10
32/32 [==============================] - 0s 7ms/step - loss: 0.2079 - sparse_categorical_accuracy: 0.9570 - val_loss: 0.4243 - val_sparse_categorical_accuracy: 0.8710

Epoch 00004: saving model to training_1\cp.ckpt
Epoch 5/10
32/32 [==============================] - 0s 8ms/step - loss: 0.1409 - sparse_cat

In [9]:
os.listdir(checkpoint_dir)

['checkpoint', 'cp.ckpt.data-00000-of-00001', 'cp.ckpt.index']

In [10]:
model = create_model()

loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 2.3550 - sparse_categorical_accuracy: 0.1120
Untrained model, accuracy: 11.20%


In [11]:
model.load_weights(checkpoint_path)

loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.3910 - sparse_categorical_accuracy: 0.8750
Restored model, accuracy: 87.50%


In [13]:
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

batch_size = 32

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=5*batch_size)

model = create_model()

model.save_weights(checkpoint_path.format(epoch=0))

model.fit(train_images,
         train_labels,
         epochs=50,
         batch_size=batch_size,
         callbacks=[cp_callback],
         validation_data=(test_images, test_labels),
         verbose=0)


Epoch 00005: saving model to training_2\cp-0005.ckpt

Epoch 00010: saving model to training_2\cp-0010.ckpt

Epoch 00015: saving model to training_2\cp-0015.ckpt

Epoch 00020: saving model to training_2\cp-0020.ckpt

Epoch 00025: saving model to training_2\cp-0025.ckpt

Epoch 00030: saving model to training_2\cp-0030.ckpt

Epoch 00035: saving model to training_2\cp-0035.ckpt

Epoch 00040: saving model to training_2\cp-0040.ckpt

Epoch 00045: saving model to training_2\cp-0045.ckpt

Epoch 00050: saving model to training_2\cp-0050.ckpt


In [14]:
os.listdir(checkpoint_dir)

['checkpoint',
 'cp-0000.ckpt.data-00000-of-00001',
 'cp-0000.ckpt.index',
 'cp-0005.ckpt.data-00000-of-00001',
 'cp-0005.ckpt.index',
 'cp-0010.ckpt.data-00000-of-00001',
 'cp-0010.ckpt.index',
 'cp-0015.ckpt.data-00000-of-00001',
 'cp-0015.ckpt.index',
 'cp-0020.ckpt.data-00000-of-00001',
 'cp-0020.ckpt.index',
 'cp-0025.ckpt.data-00000-of-00001',
 'cp-0025.ckpt.index',
 'cp-0030.ckpt.data-00000-of-00001',
 'cp-0030.ckpt.index',
 'cp-0035.ckpt.data-00000-of-00001',
 'cp-0035.ckpt.index',
 'cp-0040.ckpt.data-00000-of-00001',
 'cp-0040.ckpt.index',
 'cp-0045.ckpt.data-00000-of-00001',
 'cp-0045.ckpt.index',
 'cp-0050.ckpt.data-00000-of-00001',
 'cp-0050.ckpt.index']

In [15]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2\\cp-0050.ckpt'

In [16]:
model = create_model()

model.load_weights(latest)

loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%". format(100 * acc))

32/32 - 0s - loss: 0.4868 - sparse_categorical_accuracy: 0.8740
Restored model, accuracy: 87.40%


In [18]:
model.save_weights('./checkpoints/my_checkpoint')

model = create_model()

model.load_weights('./checkpoints/my_checkpoint')

loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 2.3558 - sparse_categorical_accuracy: 0.1020
Restored model, accuracy: 10.20%


In [24]:
model = create_model()
model.fit(train_images, train_labels, epochs=5)

!mkdir -p saved_model
model.save('saved_model/my_model')



Epoch 1/5
32/32 [==============================] - 1s 7ms/step - loss: 1.5857 - sparse_categorical_accuracy: 0.5126
Epoch 2/5
32/32 [==============================] - 0s 8ms/step - loss: 0.4260 - sparse_categorical_accuracy: 0.8775
Epoch 3/5
32/32 [==============================] - 0s 7ms/step - loss: 0.3131 - sparse_categorical_accuracy: 0.9243
Epoch 4/5
32/32 [==============================] - 0s 7ms/step - loss: 0.2142 - sparse_categorical_accuracy: 0.9620
Epoch 5/5
32/32 [==============================] - 0s 7ms/step - loss: 0.1631 - sparse_categorical_accuracy: 0.9639


A subdirectory or file -p already exists.
Error occurred while processing: -p.
A subdirectory or file saved_model already exists.
Error occurred while processing: saved_model.


INFO:tensorflow:Assets written to: saved_model/my_model\assets


INFO:tensorflow:Assets written to: saved_model/my_model\assets


In [25]:
model = create_model()
model.fit(train_images, train_labels, epochs=5)

model.save('my_model.h5')

Epoch 1/5
32/32 [==============================] - 0s 4ms/step - loss: 1.6691 - sparse_categorical_accuracy: 0.4726
Epoch 2/5
32/32 [==============================] - 0s 4ms/step - loss: 0.4521 - sparse_categorical_accuracy: 0.8754
Epoch 3/5
32/32 [==============================] - 0s 4ms/step - loss: 0.3072 - sparse_categorical_accuracy: 0.9129
Epoch 4/5
32/32 [==============================] - 0s 5ms/step - loss: 0.2160 - sparse_categorical_accuracy: 0.9471
Epoch 5/5
32/32 [==============================] - 0s 4ms/step - loss: 0.1382 - sparse_categorical_accuracy: 0.9727


In [26]:
new_model = tf.keras.models.load_model('my_model.h5')

new_model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_8 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_17 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [27]:
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

32/32 - 0s - loss: 0.3960 - sparse_categorical_accuracy: 0.8690
Restored model, accuracy: 86.90%
